In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
from sklearn.naive_bayes import BernoulliNB
import torch
import transformers as ppb
import warnings
import joblib

In [2]:
df = pd.read_csv('data_crawler.csv', delimiter='\t', header=None)
df


,0
0,"Text,""Sentiment"""
1,"ok,2"
2,máy mới sạc chưa sài mà đã hết pin .cảm ứng rấ...
3,"Máy bền, bé làm rơi nhiều lần mà không bị hỏng.,2"
4,Nhưng khi chơi game nhẹ thì OK game nặng giật ...
...,...
148,"Máy nhẹ, RAM 1.5GB đủ dùng cho các tác vụ nhẹ...."
149,Sử dụng tốt với các nhu cầu cơ bản như xem You...
150,Tốt trong tầm giá. Máy mới nguyên seal. Mọi th...
151,"Dùng rất tốt, máy nguyên seal và đóng gói cực ..."


In [3]:
batch_1 = df
batch_1[0].value_counts()

Máy tốt, giao hàng trước thời hạn dự kiến. Nhưng có 2 vấn đề:\n- Máy không gọi điện được mặc dù xem phần cài đặt có thấy cài đặt gọi điện.\n- Tai nghe âm lượng rất nhỏ, không thể sử dụng đươc.\nĐề nghị nhà cung cấp xem xét.,0    2
May tam on, nhung nhanh het pin, su dung khoang nua tieng la may nong qua.Minh muon doi may khac nhe,0                                                                                                                               2
ok,2                                                                                                                                                                                                                                 2
Đặt hàng Huawei nhưng giao Lenovo T9S giá còn không tương đương.\nĐã vậy còn không cho kiểm hàng.,0                                                                                                                                  2
Text,"Sentiment"                                                            

In [4]:
batch_1 = df
# print(df)

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
# joblib.dump(model, 'tfidf.pkl')
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized


0              [101, 3793, 1010, 1000, 15792, 1000, 102]
1                           [101, 7929, 1010, 1016, 102]
2      [101, 2089, 25175, 17266, 14684, 2050, 18952, ...
3      [101, 2089, 3841, 1010, 2022, 16983, 25223, 18...
4      [101, 18699, 5575, 1047, 4048, 18151, 2208, 18...
                             ...                        
148    [101, 2089, 18699, 2063, 1010, 8223, 1015, 101...
149    [101, 10514, 29328, 2000, 2102, 29536, 2072, 6...
150    [101, 2000, 2102, 19817, 5063, 17214, 27699, 1...
151    [101, 29328, 9350, 2000, 2102, 1010, 2089, 165...
152    [101, 10514, 2050, 21766, 2050, 13523, 18699, ...
Name: 0, Length: 153, dtype: object

In [5]:
batch_1[0][1][-1]

'2'

In [6]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape
 

(153, 139)

In [7]:
# print(len(batch_1[0]))
labels = np.zeros(len(batch_1[0]))
for i in range(1,len(batch_1[0])):
  labels[i]=batch_1[0][i][-1]
  
# print(labels.shape)

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
# for fe in features:
#   print(fe)


In [8]:
lr_clf = BernoulliNB(binarize = .5)
lr_clf.fit(train_features, train_labels)


BernoulliNB(binarize=0.5)

In [10]:
lr_clf

BernoulliNB(binarize=0.5)

In [9]:
joblib.dump(lr_clf, 'saved_model.pkl')
sc=lr_clf.score(test_features, test_labels)
print("score: ", sc )

score:  0.6410256410256411
